In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
pd.options.plotting.backend = 'plotly'

from tokenizers import Tokenizer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

dataframes:dict[str, pd.DataFrame] = {}
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        dataframes[filename.split(".")[0]] = pd.read_csv(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/prompts_train.csv
../data/prompts_test.csv
../data/summaries_train.csv
../data/summaries_test.csv
../data/sample_submission.csv


**Task** : 
- For a given prompt text and prompt question, predict content and wording score.
- Test data contain new text, new prompt and new summaries

**Battle plan:**
- Embed summaries 
- Embed prompts 
- fit a regressor on content score and on wording score.

In [2]:
(sample_submission, prompts_train, summaries_test, summaries_train, prompts_test) = (
    dataframes['sample_submission'], dataframes['prompts_train'], dataframes['summaries_test'], 
    dataframes['summaries_train'], dataframes['prompts_test'], 
)
full_train = summaries_train.merge(prompts_train, on="prompt_id", how='left')
full_test = summaries_test.merge(prompts_test, on="prompt_id", how='left')

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer:Tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)



/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
training_prompt_ids = ['814d6b','3b9047']
train = full_train.loc[full_train['prompt_id'].isin(training_prompt_ids)]
validate =  full_train.loc[~full_train['prompt_id'].isin(training_prompt_ids)]
train.shape

(3112, 8)

In [5]:
def get_tokens(df:pd.DataFrame)->torch.Tensor:
    prompt_tokens = tokenizer(df['prompt_question'].tolist(),  padding='max_length', truncation=True,  return_tensors="pt")
    summaries_tokens = tokenizer(df['text'].tolist(), padding='max_length', truncation=True,  return_tensors="pt")
    tokens = torch.cat([summaries_tokens['input_ids'],prompt_tokens['input_ids']], dim=1)
    return tokens
    

In [6]:
train_tokens = get_tokens(train)

In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

#mlp = MLPRegressor(hidden_layer_sizes=(100,))
content_rf = RandomForestRegressor()
wording_rf = RandomForestRegressor()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
content_rf.fit(train_tokens, train['content'])
wording_rf.fit(train_tokens, train['wording'])

/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated 

RandomForestRegressor()

In [9]:
validate['content_predicted'] = content_rf.predict(get_tokens(validate))
validate['wording_predicted'] = wording_rf.predict(get_tokens(validate))

/var/folders/w6/9vzp1n7s0n77fnqjpvnhtjtc0000gn/T/ipykernel_7178/990504307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate['content_predicted'] = content_rf.predict(get_tokens(validate))
/var/folders/w6/9vzp1n7s0n77fnqjpvnhtjtc0000gn/T/ipykernel_7178/990504307.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate['wording_predicted'] = wording_rf.predict(get_tokens(validate))


In [10]:
validate.plot.scatter(x="content", y='content_predicted')

In [11]:
validate.plot.scatter(x="wording", y='wording_predicted')

# -1. Submission

In [12]:
full_test

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...
1,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...
2,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...


In [14]:
wording_rf.fit(get_tokens(full_train), full_train['wording'])
content_rf.fit(get_tokens(full_train), full_train['content'])

/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/lambertbruyas/miniconda3/envs/hugging_face/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.



RandomForestRegressor()

In [ ]:
full_test['wording'] = wording_rf.predict(get_tokens(full_test))
full_test['content'] = content_rf.predict(get_tokens(full_test))

: 

In [ ]:
sample_submission = full_test[['student_id', 'content', 'wording']]
sample_submission

,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


: 

: 